In [10]:
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
import anywidget
import traitlets
import jupyter
import tweet_browser as tb
from matplotlib import pyplot as plt
from IPython.display import display, Javascript
import pandas as pd
import numpy as np
import io
import math
import time
import json
import time
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from custom_widgets import *
from ui_modules import *
import asyncio
import nest_asyncio
nest_asyncio.apply()

TWEETS_PER_PAGE = 20
# DEBUG_MODE = True
# # JUPYTER_FILE_PATH = "../tree/images/"
# JUPYTER_FILE_PATH = "images/"

out = widgets.Output()

fileUp = widgets.widgets.FileUpload(
    accept='.csv, .txt, .xls, .tsv',
    multiple=False,
    description='Change Dataset'
)

dummyEl = DummyElement()
advancedPage = AdvancedPage()
randomSampleTab = RandomSampleTab()
filterModule = FilterModule()
AiSummary = AISummaryModule()
stanceModule = StanceAnalysisModule()
TimeSeries = TimeSeriesModule()

def startSession(file):
    if file['type'] == 'xls':
        data = pd.read_excel(io.BytesIO(file.content))
    else:
        data = pd.read_csv(io.BytesIO(file.content))
    s = tb.Session(data, False)
    dummyEl.fileName = file['name']
    browser = Browser(s, out)

def autoStartSession(fileName):
    data = tb.parse_data(fileName)
    # s = tb.Session(data, False)
    s = tb.Session(data, False, embeddings=pd.read_csv("allCensus_sample_embeddings.csv", encoding = "utf-8", index_col=0))
    dummyEl.fileName = fileName
    browser = Browser(s, out)

# def selectColumns (row, colHeaders: list):
#     result = []
#     for j in colHeaders:
#         result.append(row[s.headerDict[j]])
#     return result
    
class Browser:
    def __init__(self, s, out):
        self.s = s
        dummyEl.size = s.length
        dummyEl.observe(self.alertHandler, ["changeSignal"])
        dummyEl.observe(self.searchAlertHandler, ["searchChangeSignal"])
        dummyEl.observe(self.filterAlertHandler, ["filterChangeSignal"])
        minDate = self.s.findMinDate().strftime("%Y-%m-%d")
        maxDate = self.s.findMaxDate().strftime("%Y-%m-%d")
        dummyEl.calendarStart = minDate
        dummyEl.calendarEnd = maxDate
        self.stanceResultsValid = False
        self.stanceAnalysisTask = None
        self.screen = "main"
        self.colHeaders = list(s.headerDict.keys())
        self.createWidgets()
        self.search()
        self.resetDisplay()
        # self.history = [StoredSearch()]

### Widgets
    def createWidgets(self):
        self.datasetDisplay = fileUp
        self.advancedButton = widgets.Button(description='Click here to enter search query').add_class("long-button")
        self.advancedButton.on_click(self.openSearchMenu)
        self.searchedCriteria = widgets.HTML("SEARCHED CRITERIA", layout=widgets.Layout(padding="0px 32px")).add_class("heading4")
        self.advancedBar = widgets.VBox([self.searchedCriteria, self.advancedButton]).add_class("advanced-bar")

        self.makeRandomSampleTab()
        self.makeCentralityTab()
        self.makeAiSummaryPage()
        self.makeFilterBar()
        self.makeStanceAnalysisPage()
        self.makeTimeSeriesPage()
        
        self.tabs = widgets.Tab(children=[randomSampleTab.randomSelection, self.centralTweetBox, self.summaryTab, self.stanceAnalysis, TimeSeries.timeSeries], titles=["Random Posts", "Typical Posts", "AI Summary", "Stance Analysis", "Time Series"])
        self.tabs.observe(self.loadTab, names=["selected_index"])
        self.topBar = widgets.HBox([widgets.HTML("Social Media Browser").add_class("title"), self.datasetDisplay]).add_class("top-bar")
        self.mainPage = widgets.VBox([self.topBar, self.advancedBar, widgets.HBox([self.filterBox, self.tabs])])
        # self.mainPage = widgets.VBox([self.paramDisplay, self.tabs])
        self.debugText = widgets.HTML("test")
        self.makeAdvancedPage()

    def makeRandomSampleTab(self):
        randomSampleTab.tweetDisplay.observe(self.getTweets, names=["pageNum"])
        randomSampleTab.sortBar.observe(self.getTweets, names=["sortScope", "sortColumn", "sortOrder"])
        randomSampleTab.sampleSelector.observe(self.generateNewSample, names=["changeSignal"])
        self.randomSelection = randomSampleTab.randomSelection

    def makeCentralityTab(self):
        # self.typicalSampleTitle = widgets.HTML().add_class("display-count")
        self.typicalSampleTitle = TypicalSampleSelector()
        self.typicalSampleTitle.observe(self.getTypicalPosts, ["changeSignal"])
        self.displayCentralityScore = ToggleSwitch(label = "Typicality", value=0).add_class("tweet-display-add-on")
        self.displayCentralityScore.observe(self.toggleTypicalityScore, ["value"])
        self.centralTweets = TweetDisplay(height="60vh", displayAddOn=0, addOnColumnName="centrality")
        self.centralTweetBox = widgets.VBox([widgets.HBox([self.typicalSampleTitle, self.displayCentralityScore], layout=widgets.Layout(flex="0 0", margin="0px 0px 16px 0px")), self.centralTweets], layout=widgets.Layout(max_height="100%"))
        
    def makeFilterBar(self):
        filterModule.mainPageClear.on_click(self.clearFilters)
        filterModule.mainPageSearch.on_click(self.startFilter)
        filterModule.userName.observe(self.displayPopUp, names=["value"])
        filterModule.geography.observe(self.displayPopUp, names=["value"])
        filterModule.allowRetweets.observe(self.displayPopUp, names=["value"])
        filterModule.weightBy.observe(self.displayPopUp, names=["value"])
        filterModule.fromDate.observe(self.displayPopUp, names=["value"])
        filterModule.toDate.observe(self.displayPopUp, names=["value"])
        self.filterBox = filterModule.filterBox

    def makeAdvancedPage(self):
        advancedPage.hiddenButton.on_click(self.startSearch)
        advancedPage.clearButton.on_click(self.clearSearch)
        advancedPage.closeButton.on_click(self.closeSearchMenu)
        self.advancedPage = advancedPage.advancedPage
    
    def makeAiSummaryPage(self):
        AiSummary.aiSummary.observe(self.updateAiPageSelect, names=["changeSignal"])
        AiSummary.pageSelect.observe(self.getSummaryTweets, names=["value"])
        AiSummary.newSummaryButton.on_click(self.generateNewSummary)
        self.summaryTab = AiSummary.summaryTab

    def makeStanceAnalysisPage(self):
        self.stanceAnalysis = stanceModule.stanceAnalysis
        self.stanceAnalysis.observe(self.showStanceAnalysisResults, names=["pageNumber"])
        self.stanceAnalysis.observe(self.openSearchMenu, names=["changeSignal"])
        stanceModule.modifyStanceButton.on_click(self.backToStancePage)
        stanceModule.sampleSelector.observe(self.showStanceAnalysisResults, names=["changeSignal"])
        for cb in stanceModule.checkboxes:
            cb.observe(self.getStanceTweets, names=["value"])
        stanceModule.sortBar.observe(self.getStanceTweets, names=["sortScope", "sortColumn", "sortOrder"])
        stanceModule.tweetDisplay.observe(self.getStanceTweets, names=["pageNum"])
        stanceModule.tweetDisplay.observe(self.addStanceCorrection, names=["newStanceCorrectionNum"])
        stanceModule.cancelModification.on_click(self.cancelStanceCorrection)
        stanceModule.retrainButton.on_click(self.retrain)
        self.stanceAnalysisPage = self.stanceAnalysis

    def makeTimeSeriesPage(self):
        TimeSeries.timeSeriesMode.observe(self.generateGraph, names=["value"])
        for i in range(len(TimeSeries.genderCheckboxes)):
            checkbox = TimeSeries.genderCheckboxes[i]
            checkbox.add_class("time-checkbox"+str(i))
            checkbox.observe(self.generateGraph, names=["value"])
        for i in range(4):
            temp = widgets.Checkbox(value=True, indent=False).add_class("stance-checkbox" + str(i))
            temp.observe(self.generateGraph, names=["value"])
            TimeSeries.stanceCheckboxes.append(temp)
        temp = widgets.Checkbox(value=True, indent=False, description="System Default - Irrelevant").add_class("stance-checkbox-1")
        temp.observe(self.generateGraph, names=["value"])
        TimeSeries.stanceCheckboxes.append(temp)
        
        
### Control flow
    def resetDisplay(self, b = None):
        out.clear_output(True)
        with out:
            if self.screen == "main":
                display(self.mainPage)
                self.loadTab(None, self.tabs.selected_index)
                self.getTweets()
            elif self.screen == "advanced":
                display(self.advancedPage)
            # display(self.debugText)
            display(dummyEl)

    def alertHandler(self, change):
        if dummyEl.userResponse == 1:
            self.search()
            self.loadTab(None, self.tabs.selected_index)
        dummyEl.userResponse = 2

    def loadTab(self, change, tabNum = None):
        if change != None:
            tabNum = change["new"]
            if self.hasChanges:
                # dummyEl.alertTrigger += 1
                with out:
                    display(Javascript('window.confirmChanges()'))
        if tabNum == 1:
            self.getTypicalPosts()
        elif tabNum == 2:
            self.generateSummary()
        elif tabNum == 4:
            self.generateGraph()

    def openSearchMenu(self, change):
        self.screen = "advanced"
        self.resetDisplay()

    def displayPopUp(self, change=None):
        self.hasChanges = True
        filterModule.filterBox.children = [filterModule.mainFilters, filterModule.popUpOptions]

### Keyword search      
    def startSearch(self, change=None):
        if dummyEl.activeStanceAnalysis == 1:
            # dummyEl.searchTrigger += 1
            with out:
                display(Javascript('window.confirmSearch()'))
        else:
            self.search()

    def startFilter(self, change=None):
        # dummyEl.filterTrigger += 1
        with out:
            display(Javascript('window.confirmFilter()'))
        
    def searchAlertHandler(self, change=None):
        if dummyEl.userResponse == 1:
            self.search()
        dummyEl.userResponse = 2

    def filterAlertHandler(self, change=None):
        if dummyEl.userResponse == 1:
            self.search()
        dummyEl.userResponse = 2
    
    def search(self, b=None):
        self.s.currentSet = self.s.base
        randomSampleTab.displaySimilarityScore.hidden = 1
        usedKeywordSearch = False
        self.stanceResultsValid = False
        if self.stanceAnalysisTask is not None:
            self.stanceAnalysisTask.cancel()
        for i in range(filterModule.geography.count):
            self.s.filterBy("State", json.loads(filterModule.geography.value)[i].lower())
        for i in range(filterModule.userName.count):
            self.s.filterBy("SenderScreenName", json.loads(filterModule.userName.value)[i])
        if(filterModule.fromDate.value != None and filterModule.toDate.value != None):
            self.s.filterDate(filterModule.fromDate.value.strftime('%Y-%m-%d'), filterModule.toDate.value.strftime('%Y-%m-%d'))
        if(filterModule.allowRetweets.value == 0):
            self.s.removeRetweets()
        if(advancedPage.semanticSearch.value != ""):
            usedKeywordSearch = True
            randomSampleTab.displaySimilarityScore.hidden = 0
            self.s.semanticSearch(advancedPage.semanticSearch.value, float(advancedPage.semanticSearch.filterPercent / 100))
        if(advancedPage.exclude.count > 0):
            usedKeywordSearch = True
            self.s.exclude(json.loads(advancedPage.exclude.value))
        if(advancedPage.mustInclude.count > 0):
            usedKeywordSearch = True
            self.s.searchKeyword(json.loads(advancedPage.mustInclude.value), False)
        if(advancedPage.containOneOf.count > 0):
            usedKeywordSearch = True
            self.s.searchKeyword(json.loads(advancedPage.containOneOf.value), True)
        randomSampleTab.toggleSimilarityScore()
        self.filterBox.children = [filterModule.mainFilters]
        self.hasChanges = False
        randomSampleTab.sampleSelector.total = self.s.currentSet.size
        stanceModule.sampleSelector.total = self.s.currentSet.size
        randomSampleTab.tweetDisplay.pageNum = 1
        self.currentWorkingSet = self.s.currentSet
        AiSummary.aiSummary.rerender = 1
        self.backToStancePage()
        if usedKeywordSearch == False:
            self.stanceAnalysis.pageNumber = -1
        self.updateSearchParams()

    def tryGetNewSample(self):
        randomSampleTab.tweetDisplay.pageNum = 1
        sampleSize = randomSampleTab.sampleSelector.value
        if self.s.currentSet.size < randomSampleTab.sampleSelector.value:
            randomSampleTab.sampleSelector.value = -1
        if randomSampleTab.sampleSelector.value == -1:
            sampleSize = self.s.currentSet.size
        
        if filterModule.weightBy.value == "None":
            self.s.simpleRandomSample(sampleSize)  
        else:
            self.s.weightedSample(sampleSize, filterModule.weightBy.value)
        self.getTweets()        
    
    def generateNewSample(self, b):
        # make sure to call only after a sample has already been generated
        # self.s.back()
        self.s.currentSet = self.currentWorkingSet
        self.tryGetNewSample()
    
    def getTweets(self, change=None):
        dataSet = self.s.getCurrentSubset()
        pageNum = randomSampleTab.tweetDisplay.pageNum
        
        tempArr = []
        sorted = self.getSortedTweets(pageNum)
        randomSampleTab.tweetDisplay.maxPage = math.ceil(self.s.currentSet.size / TWEETS_PER_PAGE)
        for i in range(len(sorted)):   
            tempArr.append(sorted.iloc[i].to_json())
        randomSampleTab.sampleTitle.value = "Displaying " + format(self.s.currentSet.size, ',d') + " posts from " + format(randomSampleTab.sampleSelector.total, ',d') + " results"
        randomSampleTab.tweetDisplay.value = tempArr
    
    def getSortedTweets(self, pageNum, currSet=None, sortBar=None):
        if currSet is None:
            currSet = self.s.getCurrentSubset()
        if sortBar is None:
            sortBar = randomSampleTab.sortBar
        ans = currSet
        
        asc = True
        na_pos = "last"
        keyFunc = None
            
        if sortBar.sortColumn == "Username" or sortBar.sortColumn == "SenderScreenName":
            keyFunc = userNameToLower
        if sortBar.sortOrder == "DESC":
            asc = False
            na_pos = "last"
        if sortBar.sortColumn == "None":
            # ans = ans.sample(frac = 1)
            pass
        else:
            ans = ans.sort_values(by=[sortBar.sortColumn], ascending=asc, na_position = na_pos, key=keyFunc)
        ans = ans.iloc[max((pageNum-2) * TWEETS_PER_PAGE, 0) : min(pageNum * TWEETS_PER_PAGE, len(ans))]
        return ans

    def clearSearch(self, change):
        advancedPage.semanticSearch.value = ""
        # advancedPage.semanticSearch.filterPercent = 50
        advancedPage.exclude.value = "[]"
        advancedPage.exclude.count = 0
        advancedPage.mustInclude.value = "[]"
        advancedPage.mustInclude.count = 0
        advancedPage.containOneOf.value = "[]"
        advancedPage.containOneOf.count = 0
        self.resetDisplay()
        
    def clearFilters(self, change):
        filterModule.geography.value = "[]"
        filterModule.geography.count = 0
        filterModule.userName.value = "[]"
        filterModule.userName.count = 0
        filterModule.fromDate.value = None
        filterModule.toDate.value = None
        filterModule.allowRetweets.value = True
        filterModule.weightBy.value = "None"
        self.filterBox.children = [filterModule.mainFilters]
        self.search()
        # self.resetDisplay()

    def updateSearchParams(self, change=None):
        searchedString = ""
        if len(advancedPage.semanticSearch.value) > 0:
            searchedString += advancedPage.semanticSearch.value + " "
        if(advancedPage.mustInclude.count > 0):
            searchedString += "(" + ' AND '.join(json.loads(advancedPage.mustInclude.value)) + ")"
        if(advancedPage.containOneOf.count > 0):
            if len(searchedString) > 0 and searchedString[-1] == ")":
                searchedString += " AND "
            searchedString += "(" + ' OR '.join(json.loads(advancedPage.containOneOf.value)) + ")"
        if(advancedPage.exclude.count > 0):
            if len(searchedString) > 0 and searchedString[-1] == ")":
                searchedString += " AND "
            searchedString += "NOT (" + ' OR '.join(json.loads(advancedPage.exclude.value)) + ")"
        if len(searchedString) > 0:
            self.advancedButton.description = searchedString
        else:
            self.advancedButton.description = "Click here to enter search query"
        newKeywords = ""
        if(advancedPage.mustInclude.count > 0):
            newKeywords += ','.join(json.loads(advancedPage.mustInclude.value))
        if(advancedPage.containOneOf.count > 0):
            if len(newKeywords) > 0:
                newKeywords += ","
            newKeywords += ','.join(json.loads(advancedPage.containOneOf.value))
        randomSampleTab.tweetDisplay.keywords = newKeywords
        self.closeSearchMenu(change)

    def closeSearchMenu(self, change):
        self.screen = "main"
        self.resetDisplay()
        
# Stance analysis logic
    async def startStanceAnalysis(self, keepCurrResults = False):
        sampleSize = stanceModule.sampleSelector.value
        if self.s.currentSet.size < stanceModule.sampleSelector.value:
            stanceModule.sampleSelector.value = -1
        if stanceModule.sampleSelector.value == -1:
            sampleSize = self.s.currentSet.size
        if not keepCurrResults:
            self.s.simpleRandomSample(sampleSize)
        stanceModule.loadingScreen.processInitial = sampleSize
        self.tabs.children = [*self.tabs.children[:3], stanceModule.loadingScreen, *self.tabs.children[4:]]
        stanceModule.stanceAnalysisResults.children = stanceModule.stanceAnalysisResults.children[:5]
        try:
            dummyEl.activeStanceAnalysis = 1
            newCheckboxes = []
            newStances = []
            for i in range(len(self.stanceAnalysis.stances)):
                if self.stanceAnalysis.stances[i] != "":
                    newStances.append(i)
                    stanceModule.checkboxes[i].description = "Stance " + str(i+1) + " " + self.stanceAnalysis.stances[i]
                    stanceModule.checkboxes[i].value = True
                    newCheckboxes.append(stanceModule.checkboxes[i])
            newStances.append(-1)
            stanceModule.tweetDisplay.stances = newStances
            newCheckboxes.append(stanceModule.checkboxes[-1])
            stanceModule.checkboxBar.children = newCheckboxes
            stanceExamples = {}
            for i in range(len(self.stanceAnalysis.examples)):
                if self.stanceAnalysis.examples[i] != "":
                    stanceExamples[self.stanceAnalysis.examples[i]] = i
            for i in range(len(stanceModule.stanceCorrections)):
                stanceExamples[stanceModule.stanceCorrections[i]] = stanceModule.stanceCorrectionNums[i]

            stanceModule.lastSearchedStances = self.stanceAnalysis.stances
            await self.s.stanceAnalysis(self.stanceAnalysis.topic, self.stanceAnalysis.stances, stanceExamples, updateAllData=True)
            self.getStanceTweets()
            self.stanceAnalysisPage = stanceModule.stanceAnalysisResults
            self.tabs.children = [*self.tabs.children[:3], self.stanceAnalysisPage, *self.tabs.children[4:]]
            dummyEl.activeStanceAnalysis = 0
            self.stanceResultsValid = True
        except:
            self.stanceAnalysisPage = self.stanceAnalysis
            self.tabs.children = [*self.tabs.children[:3], self.stanceAnalysisPage, *self.tabs.children[4:]]
            self.stanceAnalysis.pageNumber = 0
            dummyEl.activeStanceAnalysis = 0
            with out:
                display(Javascript('alert("Error genearting stance analysis");'))
    
    def retrain(self, change=None):
        self.showStanceAnalysisResults(keepCurrResults=True)
    
    def showStanceAnalysisResults(self, change=None, keepCurrResults = False):
        if not keepCurrResults:
            self.s.currentSet = self.currentWorkingSet
        if self.stanceAnalysis.pageNumber <= 0:
            return
        loop = asyncio.get_event_loop()
        self.stanceAnalysisTask = asyncio.create_task(self.startStanceAnalysis(keepCurrResults))
        # asyncio.run(self.startStanceAnalysis())

    def getStanceTweets(self, change=None): # TODO: refactor this into get tweets
        tempDf = self.filterStances()
        
        pageNum = stanceModule.tweetDisplay.pageNum
        tempDf["stance"] = tempDf["stance"].replace(-1, None)
        sorted = self.getSortedTweets(pageNum, tempDf, stanceModule.sortBar)
        sorted["stance"] = sorted["stance"].fillna(-1)
        stanceModule.tweetDisplay.maxPage = math.ceil(len(tempDf) / TWEETS_PER_PAGE)
        tempArr = []
        for i in range(len(sorted)):   
            tempArr.append(sorted.iloc[i].to_json())
        stanceModule.tweetDisplay.value = tempArr

    def filterStances(self, change=None, df=None):
        if df is None:
            df = self.s.getCurrentSubset().copy(deep=True)
        if stanceModule.checkboxes[-1].value == False:
            df = df[df["stance"] != -1]
        for i in range(4):
            if stanceModule.checkboxes[i].value == False:
                df = df[df["stance"] != i]
        temp = []
        for i in range(len(df)):
            temp.append(df.iloc[i].to_json())
        # stanceModule.tweetDisplay.value = temp
        stanceModule.title.value = "Displaying " + format(len(df), ',d') + " most relevant tweets from " + format(self.currentWorkingSet.size, ',d') + " results"
        return df

    def addStanceCorrection(self, change):
        stanceModule.stanceCorrections.append(stanceModule.tweetDisplay.stanceCorrection)
        stanceModule.stanceCorrectionNums.append(stanceModule.tweetDisplay.newStanceCorrectionNum)
        stanceModule.tweetDisplay.stanceCorrection = ""
        stanceModule.tweetDisplay.newStanceCorrectionNum = -2
        stanceModule.stanceAnalysisResults.children = stanceModule.stanceAnalysisResults.children + (stanceModule.popUp)
    
    def cancelStanceCorrection(self, change=None):
        stanceModule.stanceCorrections = []
        stanceModule.stanceCorrectionNums = []
        stanceModule.stanceAnalysisResults.children = stanceModule.stanceAnalysisResults.children[:5]
        self.resetDisplay()
    
    def backToStancePage(self, change=None):
        self.stanceAnalysis.pageNumber = 0
        self.stanceAnalysisPage = self.stanceAnalysis
        self.tabs.children = [*self.tabs.children[:3], self.stanceAnalysisPage, *self.tabs.children[4:]]
        self.cancelStanceCorrection()
### Typical posts logic
    def toggleTypicalityScore(self, change):
        self.centralTweets.displayAddOn = self.displayCentralityScore.value

    def getTypicalPosts(self, change=None):
        tempArr = []
        self.s.currentSet = self.currentWorkingSet
        result = self.s.getCentral()
        self.typicalSampleTitle.total = len(result)
        if len(result) == 0:
            self.typicalSampleTitle.visible = 0
            # self.typicalSampleTitle.value = ""
        else:
            self.typicalSampleTitle.visible = 1
            for i in range(self.typicalSampleTitle.value):
                tempArr.append(result.iloc[i].to_json())
        self.centralTweets.value = tempArr
### AI summary logic
    def updateAiPageSelect(self, change=None):
        AiSummary.pageSelect.value = AiSummary.aiSummary.selected + 1
        AiSummary.pageSelect.changeSignal += 1
        self.getSummaryTweets()

    def generateNewSummary(self, change=None):
        self.s.currentSet = self.currentWorkingSet
        AiSummary.aiSummary.rerender = 1
        self.generateSummary()
    
    def getSummaryTweets(self, change=None):
        pageNum = AiSummary.pageSelect.value - 1 # convert to 0 indexing
        AiSummary.aiSummary.selected = pageNum
        tweets = AiSummary.aiSummary.sentenceNums[pageNum]
        ans = self.s.allData.iloc[tweets]
        tempArr = []
        for i in range(len(ans)):
            tempArr.append(ans.iloc[i].to_json())
        AiSummary.summaryDisplay.value = tempArr
        AiSummary.pageSelect.changeSignal += 1
        # self.resetDisplay()
    
    def generateSummary(self, change = None):
        if AiSummary.aiSummary.rerender == 0:
            return
        if self.s.currentSet.size > 50:
            AiSummary.title.value = "Summarizing 50 posts sampled from " + format(self.s.currentSet.size, ',d') + " results"
            self.s.simpleRandomSample(50)
        else:
            AiSummary.title.value = "Summarizing " + format(self.s.currentSet.size, ',d') + " posts sampled from " + format(self.s.currentSet.size, ',d') + " results"
        self.tabs.children = [*self.tabs.children[:2], AiSummary.loadingPage, *self.tabs.children[3:]]
        summary = self.s.summarize()
        self.tabs.children = [*self.tabs.children[:2], self.summaryTab, *self.tabs.children[3:]]
        strings, tweets, unused = self.s.parseSummary(summary)
        AiSummary.aiSummary.value = strings
        AiSummary.aiSummary.sentenceNums = tweets
        AiSummary.aiSummary.unused = unused
        AiSummary.aiSummary.rerender = 0
        AiSummary.pageSelect.maxPage = len(tweets)
        AiSummary.aiSummary.selected = 0
        AiSummary.pageSelect.value = 1
        self.getSummaryTweets(None)

# Time series logic
    def generateGraph(self, change = None):
        graphCheckboxes = None
        if self.stanceResultsValid == False:
            TimeSeries.timeSeriesMode.options = ["Overview", "Gender"]
        else:
            TimeSeries.timeSeriesMode.options = ["Overview", "Gender", "Stance"]
        TimeSeries.timeSeries.children = [TimeSeries.timeSeriesMode]
        df = self.s.getCurrentSubset().rename(columns={"Message": "Count", "CreatedTime": "Date"})
        if TimeSeries.timeSeriesMode.value == "Overview":
            data = df.groupby(df["Date"]).agg({"Count": "count"}).reset_index()
            fig = px.line(data, x="Date", y="Count")
        else:
            categories = []
            colors = px.colors.qualitative.Plotly
            if TimeSeries.timeSeriesMode.value == "Gender":
                graphCheckboxes = widgets.HBox(children=TimeSeries.genderCheckboxes).add_class("graph-checkboxes")
                keyColumn = "SenderGender"
                for checkbox in TimeSeries.genderCheckboxes:
                    if checkbox.value == True:
                        categories.append(checkbox.description)
            if TimeSeries.timeSeriesMode.value == "Stance":
                graphCheckboxes = widgets.HBox().add_class("graph-checkboxes")
                tempCheckboxes = []
                colors = []
                # if self.stanceResultsValid == False:
                #     TimeSeries.timeSeries.children = [TimeSeries.timeSeriesMode, TimeSeries.stanceNote]
                #     return
                keyColumn = "stance"
                for i in range(len(stanceModule.lastSearchedStances)):
                    if stanceModule.lastSearchedStances[i] != "":
                        if TimeSeries.stanceCheckboxes[i].value == True:
                            categories.append(stanceModule.lastSearchedStances[i])
                            colors.append(self.stanceAnalysis.colors[i])
                        df["stance"] = df["stance"].replace(i, stanceModule.lastSearchedStances[i])
                        TimeSeries.stanceCheckboxes[i].description = stanceModule.checkboxes[i].description
                        tempCheckboxes.append(TimeSeries.stanceCheckboxes[i])
                if TimeSeries.stanceCheckboxes[-1].value == True:
                    categories.append("Irrelevant")
                    df["stance"] = df["stance"].replace(-1, "Irrelevant")
                tempCheckboxes.append(TimeSeries.stanceCheckboxes[-1])
                graphCheckboxes.children = tempCheckboxes

            colors.append("#35B8FF")
            fig = go.Figure()
            df_grouped = df.groupby(['Date', keyColumn]).size().reset_index(name='Count')
            df_pivot = df_grouped.pivot(index='Date', columns=keyColumn, values='Count').fillna(0)
            for category in categories:
                if category not in df_pivot.columns:
                    df_pivot[category] = 0
            df_pivot = df_pivot.reset_index()
            df_long = df_pivot.melt(id_vars='Date', value_vars=categories, var_name=keyColumn, value_name='Count')
            fig = px.line(df_long, x="Date", y="Count", color = keyColumn, color_discrete_sequence=colors)
        graph = go.FigureWidget(fig)
        TimeSeries.graphSubtitle.value = "Total number of posts: " + format(self.s.currentSet.size, ',d')
        if graphCheckboxes == None:
            TimeSeries.timeSeries.children = [TimeSeries.timeSeriesMode, TimeSeries.graphTitle, TimeSeries.graphSubtitle, graph]
        else:
            TimeSeries.timeSeries.children = [TimeSeries.timeSeriesMode, TimeSeries.graphTitle, TimeSeries.graphSubtitle, graphCheckboxes, graph]
    
def fileHandler(change):
    startSession(fileUp.value[0])
    
def userNameToLower(input):
    return input.str.lower()

# with out:
#     display(fileUp)

autoStartSession("allCensus_sample.csv")

fileUp.observe(fileHandler, names=["value"])

out

Output()